這篇 code 是基於前面的 code，只是增加了 "計算交易 (持有)時間" 的 code

2021.09.12

In [1]:
import pandas as pd
import plotly.graph_objects as go

import utils
import instrument

In [2]:
pair = 'EUR_USD'
granularity = 'H1'
ma_list = [16, 64]
i_pair = instrument.Instrument.get_instrument_by_name("EUR_USD")

### 從 .pkl 檔找出 OANDA 提供的資料

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [4]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2021-01-18T16:00:00.000000000Z,2605,1.20749,1.20809,1.20736,1.20767,1.20743,1.20801,1.20729,1.20761,1.20755,1.20817,1.20743,1.20773
1,2021-01-18T17:00:00.000000000Z,1554,1.20766,1.20767,1.20708,1.20749,1.20759,1.20760,1.20701,1.20742,1.20773,1.20774,1.20714,1.20756
2,2021-01-18T18:00:00.000000000Z,736,1.20750,1.20780,1.20720,1.20740,1.20743,1.20773,1.20711,1.20734,1.20758,1.20788,1.20729,1.20747
3,2021-01-18T19:00:00.000000000Z,1452,1.20742,1.20819,1.20739,1.20792,1.20735,1.20810,1.20730,1.20784,1.20748,1.20832,1.20745,1.20801
4,2021-01-18T20:00:00.000000000Z,316,1.20790,1.20808,1.20770,1.20770,1.20783,1.20800,1.20759,1.20759,1.20798,1.20819,1.20781,1.20781


### 去掉我們不需要的 columns

注意到:  
- ```time``` 是要放在 x 軸
- ```mid_o```, ```mid_l```, ```mid_h```, ```mid_c``` 是為了畫蠟燭圖

In [5]:
df_ma = df[ ['time', 'mid_o', 'mid_l', 'mid_h', 'mid_c'] ].copy()

### 計算 MA

In [6]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma['mid_c'].rolling(window=ma).mean()

### 去掉 NA 值

In [7]:
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

### 判斷是否交易

In [8]:
df_ma[ ['MA_16', 'MA_64']].head()

,MA_16,MA_64
0,1.211770,1.211451
1,1.211906,1.211538
2,1.212129,1.211643
3,1.212316,1.211745
4,1.212542,1.211845


In [9]:
df_ma['DIFF'] = df_ma['MA_16'] - df_ma['MA_64']
df_ma['PREV_DIFF'] = df_ma['DIFF'].shift(1)

In [10]:
df_ma.head()
df_ma.dropna(inplace=True)

In [11]:
df_ma.head()

,time,mid_o,mid_l,mid_h,mid_c,MA_16,MA_64,DIFF,PREV_DIFF
1,2021-01-21T08:00:00.000000000Z,1.21196,1.21149,1.21374,1.21320,1.211906,1.211538,0.000368,0.000319
2,2021-01-21T09:00:00.000000000Z,1.21319,1.21274,1.21460,1.21424,1.212129,1.211643,0.000485,0.000368
3,2021-01-21T10:00:00.000000000Z,1.21422,1.21351,1.21472,1.21392,1.212316,1.211745,0.000571,0.000485
4,2021-01-21T11:00:00.000000000Z,1.21393,1.21325,1.21462,1.21433,1.212542,1.211845,0.000697,0.000571
5,2021-01-21T12:00:00.000000000Z,1.21432,1.21415,1.21551,1.21432,1.212767,1.211949,0.000818,0.000697


In [12]:
def is_trade(row):
    if row['DIFF'] >= 0 and row['PREV_DIFF'] < 0:
        return 1
    if row['DIFF'] <= 0 and row['PREV_DIFF'] > 0:
        return -1
    else:
        return 0

In [13]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [14]:
df_ma['IS_TRADE'].describe()

count    3935.000000
mean       -0.000254
std         0.146991
min        -1.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: IS_TRADE, dtype: float64

### ```df_trades```

In [15]:
df_trades = df_ma[ df_ma['IS_TRADE'] != 0 ].copy()

### 計算損益

In [16]:
df_trades['DELTA'] = df_trades['mid_c'].diff().shift(-1)
df_trades['GAIN'] = df_trades['DELTA'] * df_trades['IS_TRADE']

In [17]:
df_trades

,time,mid_o,mid_l,mid_h,mid_c,MA_16,MA_64,DIFF,PREV_DIFF,IS_TRADE,DELTA,GAIN
58,2021-01-25T17:00:00.000000000Z,1.21368,1.21362,1.21475,1.21445,1.215794,1.215855,-0.000060,0.000096,-1,0.00165,-0.00165
90,2021-01-27T01:00:00.000000000Z,1.21624,1.21584,1.21672,1.21610,1.215634,1.215497,0.000137,-0.000083,1,-0.00531,-0.00531
101,2021-01-27T12:00:00.000000000Z,1.21164,1.21056,1.21198,1.21079,1.214941,1.215047,-0.000106,0.000171,-1,0.00273,-0.00273
153,2021-01-29T16:00:00.000000000Z,1.21492,1.21298,1.21534,1.21352,1.212039,1.211844,0.000195,-0.000022,1,-0.00596,-0.00596
173,2021-02-01T12:00:00.000000000Z,1.20818,1.20722,1.20887,1.20756,1.211494,1.211547,-0.000052,0.000272,-1,-0.00265,0.00265
...,...,...,...,...,...,...,...,...,...,...,...,...
3740,2021-08-27T02:00:00.000000000Z,1.17496,1.17494,1.17576,1.17526,1.175463,1.175465,-0.000002,0.000164,-1,0.00156,-0.00156
3744,2021-08-27T06:00:00.000000000Z,1.17617,1.17588,1.17700,1.17682,1.175588,1.175554,0.000033,-0.000045,1,-0.00170,-0.00170
3746,2021-08-27T08:00:00.000000000Z,1.17614,1.17505,1.17628,1.17512,1.175572,1.175583,-0.000010,0.000004,-1,0.00078,-0.00078
3748,2021-08-27T10:00:00.000000000Z,1.17581,1.17556,1.17638,1.17590,1.175645,1.175610,0.000035,-0.000002,1,0.01068,0.01068


---

2021.09.12 新增:  
- 將交易 (持有)期間算出來  

In [18]:
df_trades.info()
# 欲更改 time 的資料型態 (object -> num)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85 entries, 58 to 3897
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       85 non-null     object 
 1   mid_o      85 non-null     float64
 2   mid_l      85 non-null     float64
 3   mid_h      85 non-null     float64
 4   mid_c      85 non-null     float64
 5   MA_16      85 non-null     float64
 6   MA_64      85 non-null     float64
 7   DIFF       85 non-null     float64
 8   PREV_DIFF  85 non-null     float64
 9   IS_TRADE   85 non-null     int64  
 10  DELTA      84 non-null     float64
 11  GAIN       84 non-null     float64
dtypes: float64(10), int64(1), object(1)
memory usage: 8.6+ KB


In [19]:
# python 內建的時間工具
from dateutil.parser import *

In [28]:
df_trades['time'] = [parse(x) for x in df_trades['time'] ]

In [21]:
df_trades.info()
# time 現在變成 datetime 物件了

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85 entries, 58 to 3897
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype                  
---  ------     --------------  -----                  
 0   time       85 non-null     object                 
 1   mid_o      85 non-null     float64                
 2   mid_l      85 non-null     float64                
 3   mid_h      85 non-null     float64                
 4   mid_c      85 non-null     float64                
 5   MA_16      85 non-null     float64                
 6   MA_64      85 non-null     float64                
 7   DIFF       85 non-null     float64                
 8   PREV_DIFF  85 non-null     float64                
 9   IS_TRADE   85 non-null     int64                  
 10  DELTA      84 non-null     float64                
 11  GAIN       84 non-null     float64                
 12  DURATION   85 non-null     datetime64[ns, tzutc()]
dtypes: datetime64[ns, tzutc()](1), float64(10), int64

In [32]:
df_trades['DURATION'] = df_trades["time"].diff().shift(-1) # 跟 GAIN 一樣，需要上移一格

In [33]:
df_trades.head()

,time,mid_o,mid_l,mid_h,mid_c,MA_16,MA_64,DIFF,PREV_DIFF,IS_TRADE,DELTA,GAIN,DURATION
58,2021-01-25 17:00:00+00:00,1.21368,1.21362,1.21475,1.21445,1.215794,1.215855,-0.000060,0.000096,-1,0.00165,-0.00165,1 days 08:00:00
90,2021-01-27 01:00:00+00:00,1.21624,1.21584,1.21672,1.21610,1.215634,1.215497,0.000137,-0.000083,1,-0.00531,-0.00531,0 days 11:00:00
101,2021-01-27 12:00:00+00:00,1.21164,1.21056,1.21198,1.21079,1.214941,1.215047,-0.000106,0.000171,-1,0.00273,-0.00273,2 days 04:00:00
153,2021-01-29 16:00:00+00:00,1.21492,1.21298,1.21534,1.21352,1.212039,1.211844,0.000195,-0.000022,1,-0.00596,-0.00596,2 days 20:00:00
173,2021-02-01 12:00:00+00:00,1.20818,1.20722,1.20887,1.20756,1.211494,1.211547,-0.000052,0.000272,-1,-0.00265,0.00265,4 days 08:00:00


由上方圖可以看到，雖然成功算出 duration，但希望將其單位轉為小時:

In [35]:
df_trades['DURATION'] = [ x.total_seconds() / 3600 for x in df_trades['DURATION'] ]
df_trades.head()
# look at first row: 1 天 8 小時 = 32 小時 (成功轉換)

,time,mid_o,mid_l,mid_h,mid_c,MA_16,MA_64,DIFF,PREV_DIFF,IS_TRADE,DELTA,GAIN,DURATION
58,2021-01-25 17:00:00+00:00,1.21368,1.21362,1.21475,1.21445,1.215794,1.215855,-0.000060,0.000096,-1,0.00165,-0.00165,32.0
90,2021-01-27 01:00:00+00:00,1.21624,1.21584,1.21672,1.21610,1.215634,1.215497,0.000137,-0.000083,1,-0.00531,-0.00531,11.0
101,2021-01-27 12:00:00+00:00,1.21164,1.21056,1.21198,1.21079,1.214941,1.215047,-0.000106,0.000171,-1,0.00273,-0.00273,52.0
153,2021-01-29 16:00:00+00:00,1.21492,1.21298,1.21534,1.21352,1.212039,1.211844,0.000195,-0.000022,1,-0.00596,-0.00596,68.0
173,2021-02-01 12:00:00+00:00,1.20818,1.20722,1.20887,1.20756,1.211494,1.211547,-0.000052,0.000272,-1,-0.00265,0.00265,104.0


---

### 畫出某些特定時段的 candle

In [24]:
df_plot = df_ma.iloc[50:120].copy()

In [25]:
df_plot.shape

(70, 10)

In [26]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()

In [2]:
import pandas as pd

In [11]:
df_all = pd.read_pickle("all_trades.pkl")

In [12]:
df_all.columns

Index(['time', 'mid_c', 'DIFF', 'DIFF_PREV', 'IS_TRADE', 'DELTA', 'GAIN',
       'PAIR', 'MASHORT', 'MALONG', 'DURATION'],
      dtype='object')

In [13]:
df_all.head()

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION
18,2021-01-19 09:00:00+00:00,1.36076,-0.000011,0.000073,-1,8.9,-8.9,GBP_USD,4,8,1.0
19,2021-01-19 10:00:00+00:00,1.36165,0.000044,-0.000011,1,20.7,20.7,GBP_USD,4,8,28.0
47,2021-01-20 14:00:00+00:00,1.36372,-0.001062,0.000375,-1,18.0,-18.0,GBP_USD,4,8,7.0
54,2021-01-20 21:00:00+00:00,1.36552,0.000424,-0.000119,1,71.8,71.8,GBP_USD,4,8,17.0
71,2021-01-21 14:00:00+00:00,1.37270,-0.000293,0.000066,-1,-16.6,16.6,GBP_USD,4,8,3.0


In [14]:
df_all.shape

(65630, 11)

In [15]:
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MASHORT,MALONG,DURATION
count,65630.000000,6.563000e+04,6.563000e+04,65630.000000,65630.000000,65630.000000,65630.000000,65630.000000,65630.000000
mean,32.055304,2.635198e-05,-1.907686e-05,-0.003139,1.283709,-1.726025,11.967210,62.011032,50.272589
std,51.591227,1.358976e-02,1.318118e-02,1.000003,51.824802,51.811956,12.671175,65.104117,77.520240
min,0.625440,-2.588750e-01,-2.635156e-01,-1.000000,-548.600000,-375.900000,4.000000,8.000000,1.000000
25%,0.913920,-1.881250e-04,-1.849870e-04,-1.000000,-19.000000,-25.200000,4.000000,16.000000,6.000000
50%,1.387770,-2.864583e-07,6.250000e-07,-1.000000,0.900000,-8.800000,8.000000,32.000000,18.000000
75%,77.726000,1.827734e-04,1.886198e-04,1.000000,20.800000,7.600000,16.000000,96.000000,68.000000
max,156.005000,2.528021e-01,2.338594e-01,1.000000,557.000000,557.000000,64.000000,256.000000,1311.000000


這麼多貨幣對與這麼多 MA 組合的策略總共交易 65,630 次  
其中，平均持有約 50 小時，持有最久的一筆交易為 1311 小時 (約 54 天)，持有最短的交易為 1 小時。

In [16]:
long_trade = df_all[ df_all['DURATION'] == 1311 ]

In [17]:
long_trade 

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION
1349,2021-04-06 20:00:00+00:00,0.93084,-0.000124,0.000071,-1,-307.4,307.4,USD_CHF,64,256,1311.0


In [22]:
short_trade = df_all[ df_all['DURATION'] == 1 ]
short_trade['PAIR'].describe()

count        3376
unique         21
top       CHF_JPY
freq          196
Name: PAIR, dtype: object

In [38]:
short_trade['PAIR'].unique()

array(['GBP_USD', 'GBP_CAD', 'GBP_JPY', 'GBP_NZD', 'GBP_CHF', 'EUR_GBP',
       'EUR_USD', 'EUR_CAD', 'EUR_JPY', 'EUR_NZD', 'EUR_CHF', 'USD_CAD',
       'USD_JPY', 'USD_CHF', 'CAD_JPY', 'CAD_CHF', 'NZD_USD', 'NZD_CAD',
       'NZD_JPY', 'NZD_CHF', 'CHF_JPY'], dtype=object)

In [57]:
short_trade.index

Int64Index([  18,   74,  260,  463,  474,  500,  555,  759,  802,  806,
            ...
            2499, 2876, 3048, 3796, 1222, 2382, 3066,  903, 3066, 3222],
           dtype='int64', length=3376)

In [68]:
short_trade.loc[74, 'PAIR']

'GBP_USD'

In [72]:
print(short_trade[ short_trade['PAIR'] == 'GBP_USD'].count()[0])

161


In [75]:
short_pairs = short_trade['PAIR'].unique()
res = {}

for p in short_pairs:
    cnt = short_trade[ short_trade['PAIR'] == p ].count()[0]
    res[p] = cnt

In [76]:
res

{'GBP_USD': 161,
 'GBP_CAD': 171,
 'GBP_JPY': 138,
 'GBP_NZD': 196,
 'GBP_CHF': 174,
 'EUR_GBP': 177,
 'EUR_USD': 158,
 'EUR_CAD': 162,
 'EUR_JPY': 167,
 'EUR_NZD': 156,
 'EUR_CHF': 156,
 'USD_CAD': 144,
 'USD_JPY': 149,
 'USD_CHF': 169,
 'CAD_JPY': 128,
 'CAD_CHF': 132,
 'NZD_USD': 143,
 'NZD_CAD': 175,
 'NZD_JPY': 154,
 'NZD_CHF': 170,
 'CHF_JPY': 196}

持有時間最長的為 USDCHF  
持有時間最短的有多個貨幣對，注意到出現 USD 的次數不多

### 遇到的問題

- https://stackoverflow.com/questions/53830081/python-pandas-the-truth-value-of-a-series-is-ambiguous